PART I

In [1]:
import re
import numpy as np
from copy import deepcopy

In [2]:
input = "input.txt"
with open(input, 'r') as infile:
    data = [l.strip() for l in infile.readlines()]

In [3]:
instructions = []
instruction_regex = re.compile('([RLDU]) (\d+) \((.+)\)')
for line in data:
    instruction_match = re.fullmatch(instruction_regex, line)
    direction, distance, color = instruction_match.groups()
    instructions.append((direction, int(distance), color))

In [30]:
#Estimate size of trench (add right-s and down-s)
#YES, I could do this in one line. But I'm lazy.
n_right = sum(distance for direction, distance, color in instructions if direction == 'R')
n_down = sum(distance for direction, distance, color in instructions if direction == 'D')
n_left = sum(distance for direction, distance, color in instructions if direction == 'L')
n_up = sum(distance for direction, distance, color in instructions if direction == 'U')

In [31]:
#Initialize trench map
trench_map = np.full((n_up + n_down, n_left + n_right), fill_value='.')
trench_map

array([['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ...,
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.']], dtype='<U1')

In [35]:
#Fill trench map
i,j = n_left, n_up
trench_map[i,j] = '#'
for direction, distance, _ in instructions:
    if direction == 'R':
        dig_sequence = [(i,j+1+step) for step in range(distance)]
    elif direction == 'D':
        dig_sequence = [(i+1+step,j) for step in range(distance)]
    elif direction == 'L':
        dig_sequence = [(i,j-1-step) for step in range(distance)]
    elif direction == 'U':
        dig_sequence = [(i-1-step,j) for step in range(distance)]
    for inew, jnew in dig_sequence:
        trench_map[inew, jnew] = '#'
    i,j = dig_sequence[-1]

In [36]:
trench_map

array([['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ...,
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.']], dtype='<U1')

In [37]:
(trench_map == '#').sum()

4292

Okay, now we need to fill that. Could apply same strategy as on Day 10:  
- For each column, go from bottom to top
- If you hit a '#', check which direction it comes from
- Once you leave the '#', check which direction they ended, if it's different, then you're 'in' - color the tiles until you hit the next '#'

In [38]:
trench_map_filled = trench_map.copy()
for j in range(len(trench_map[0])):
    inside = False
    within_trench = False
    for i in range(len(trench_map)-1,-1,-1):
        if trench_map[i,j] == '#':
            if not within_trench:
                within_trench = True
                if j>0 and trench_map[i,j-1] == '#':
                    from_left = True
                else:
                    from_left = False
            #Always record this, then we 'know' it already the moment we exit the trench trail
            if j<len(trench_map[0])-1 and trench_map[i,j+1] == '#':
                to_right = True
            else:
                to_right = False
        else:
            if within_trench:
                within_trench = False
                #Emulate an XNOR Gate (true if both or none are true)
                if from_left + to_right != 1:
                    inside = not inside
            if inside:
                trench_map_filled[i,j] = '#'

In [39]:
trench_map_filled

array([['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ...,
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.']], dtype='<U1')

In [40]:
(trench_map_filled == '#').sum()

58550

In [41]:
np.savetxt('trenchmap.txt', trench_map, fmt='%s')
np.savetxt('trenchmapfilled.txt', trench_map_filled, fmt='%s')

PART II

In [1]:
import re
import numpy as np

In [66]:
input = "input.txt"
with open(input, 'r') as infile:
    data = [l.strip() for l in infile.readlines()]

In [67]:
def decode_direction(d):
    return 'RDLU'[d]

In [68]:
instructions = []
instruction_regex = re.compile('[RLDU] \d+ \(#([0-9a-f]{5})(\d)\)')
for line in data:
    instruction_match = re.fullmatch(instruction_regex, line)
    distance, direction = instruction_match.groups()
    instructions.append((int(distance, 16), decode_direction(int(direction))))

In [69]:
instructions

[(143433, 'L'),
 (74263, 'U'),
 (207382, 'L'),
 (56659, 'U'),
 (87183, 'L'),
 (11575, 'U'),
 (224948, 'L'),
 (278117, 'U'),
 (64334, 'L'),
 (104484, 'U'),
 (376465, 'R'),
 (106516, 'U'),
 (202496, 'L'),
 (96140, 'D'),
 (272984, 'L'),
 (58530, 'D'),
 (52938, 'L'),
 (254771, 'D'),
 (52938, 'R'),
 (196535, 'D'),
 (86739, 'L'),
 (26194, 'D'),
 (282156, 'L'),
 (37936, 'U'),
 (75610, 'L'),
 (21817, 'U'),
 (206768, 'L'),
 (270386, 'U'),
 (128927, 'L'),
 (180383, 'U'),
 (335695, 'R'),
 (121648, 'U'),
 (82930, 'L'),
 (315183, 'U'),
 (178749, 'R'),
 (126169, 'U'),
 (185160, 'L'),
 (227938, 'U'),
 (185160, 'R'),
 (126332, 'U'),
 (19185, 'R'),
 (356806, 'D'),
 (137751, 'R'),
 (165079, 'U'),
 (87719, 'R'),
 (33244, 'D'),
 (104873, 'R'),
 (350211, 'D'),
 (96631, 'R'),
 (72420, 'D'),
 (89507, 'R'),
 (186790, 'U'),
 (45673, 'R'),
 (249160, 'U'),
 (45673, 'L'),
 (19925, 'U'),
 (147491, 'R'),
 (165079, 'D'),
 (53690, 'R'),
 (313083, 'U'),
 (17231, 'R'),
 (43723, 'U'),
 (322443, 'R'),
 (154777, 'U'),
 (9

I had to Google this one... quickly found out that it can be solved using the Shoelace Method: https://www.youtube.com/watch?v=FSWPX0XB7a0  
So, create a list of 2D Coordinates:

In [70]:
current_i, current_j = 1,1
instructions_with_coordinates = [(current_i, current_j)]
for distance, direction in instructions:
    if direction == 'R':
        instructions_with_coordinates.append((current_i, (current_j:=current_j+distance)))
    elif direction == 'D':
        instructions_with_coordinates.append(((current_i:=current_i+distance), current_j))
    elif direction == 'L':
        instructions_with_coordinates.append((current_i, (current_j:=current_j-distance)))
    elif direction == 'U':
        instructions_with_coordinates.append(((current_i:=current_i-distance), current_j))
        

In [71]:
instructions_with_coordinates

[(1, 1),
 (1, -143432),
 (-74262, -143432),
 (-74262, -350814),
 (-130921, -350814),
 (-130921, -437997),
 (-142496, -437997),
 (-142496, -662945),
 (-420613, -662945),
 (-420613, -727279),
 (-525097, -727279),
 (-525097, -350814),
 (-631613, -350814),
 (-631613, -553310),
 (-535473, -553310),
 (-535473, -826294),
 (-476943, -826294),
 (-476943, -879232),
 (-222172, -879232),
 (-222172, -826294),
 (-25637, -826294),
 (-25637, -913033),
 (557, -913033),
 (557, -1195189),
 (-37379, -1195189),
 (-37379, -1270799),
 (-59196, -1270799),
 (-59196, -1477567),
 (-329582, -1477567),
 (-329582, -1606494),
 (-509965, -1606494),
 (-509965, -1270799),
 (-631613, -1270799),
 (-631613, -1353729),
 (-946796, -1353729),
 (-946796, -1174980),
 (-1072965, -1174980),
 (-1072965, -1360140),
 (-1300903, -1360140),
 (-1300903, -1174980),
 (-1427235, -1174980),
 (-1427235, -1155795),
 (-1070429, -1155795),
 (-1070429, -1018044),
 (-1235508, -1018044),
 (-1235508, -930325),
 (-1202264, -930325),
 (-1202264, -8

All values need to be positive (I guess?). So shift all coordinates so that the minima are at 1 and 1:

In [72]:
min_i = min(i for i,j in instructions_with_coordinates)
min_j = min(j for i,j in instructions_with_coordinates)
instructions_with_coordinates = np.asarray([(i-min_i+1, j-min_j+1) for i,j in instructions_with_coordinates])

In [73]:
instructions_with_coordinates

array([[8859175, 1606496],
       [8859175, 1463063],
       [8784912, 1463063],
       ...,
       [9148286, 1622656],
       [9148286, 1606496],
       [8859175, 1606496]])

In [75]:
sum_one = sum(int(x)*int(y) for x,y in zip(instructions_with_coordinates[:,0], instructions_with_coordinates[1:,1]))
sum_two = sum(int(x)*int(y) for x,y in zip(instructions_with_coordinates[1:,0], instructions_with_coordinates[:,1]))

I also need to add the sum of instructions + 2 to this, because the edge of my polygon has width 1 as well! The plus 2 is the start coordinate I guess.

In [76]:
print(int(1/2*(abs(sum_one - sum_two)+(sum(i[0] for i in instructions)+2))))

47452118468566


**The approach below did not work...**

The following approach could work:
1. Iterate over instructions. Whenever one instruction overlaps with another, divide the larger one up so that there is no overlap.
2. Use this extended set of instructions to create a condensed trench map where each cell has an area assigned.
3. Perform the filling algorithm on this condensed trench map.
4. Sum up the area of each filled tile.

In [123]:
instructions_with_coordinates = []
current_i, current_j = 0, 0
for distance, direction in instructions:
    if direction == 'R':
        instructions_with_coordinates.append((direction, current_j, (current_j:=current_j+distance)))
    elif direction == 'D':
        instructions_with_coordinates.append((direction, current_i, (current_i:=current_i+distance)))
    elif direction == 'L':
        instructions_with_coordinates.append((direction, current_j-distance, current_j))
        current_j = current_j - distance
    elif direction == 'U':
        instructions_with_coordinates.append((direction, current_i-distance, current_i))
        current_i = current_i - distance
        

In [124]:
instructions_with_coordinates

[('R', 0, 461937),
 ('D', 0, 56407),
 ('R', 461937, 818608),
 ('D', 56407, 919647),
 ('R', 818608, 1186328),
 ('D', 919647, 1186328),
 ('L', 609066, 1186328),
 ('U', 356353, 1186328),
 ('L', 497056, 609066),
 ('D', 356353, 1186328),
 ('L', 5411, 497056),
 ('U', 500254, 1186328),
 ('L', 0, 5411),
 ('U', 0, 500254)]

In [125]:
def de_overlap_ranges(lower1, upper1, lower2, upper2):
    result1 = []
    result2 = []
    if lower1 < lower2:
        if upper1 <= lower2:
            result1 = [(lower1, upper1)]
            result2 = [(lower2, upper2)]
        elif upper1 < upper2:
            result1 = [(lower1,lower2),(lower2, upper1)]
            result2 = [(lower2,upper1),(upper1, upper2)]
        elif upper1 == upper2:
            result1 = [(lower1,lower2),(lower2, upper1)]
            result2 = [(lower2,upper2)]
        elif upper1 > upper2:
            result1 = [(lower1,lower2),(lower2, upper2),(upper2, upper1)]
            result2 = [(lower2,upper2)]
    elif lower1 > lower2:
        if lower1 >= upper2:
            result1 = [(lower1, upper1)]
            result2 = [(lower2, upper2)]
        elif upper1 < upper2:
            result1 = [(lower1,upper1)]
            result2 = [(lower2,lower1),(lower1, upper1), (upper1,upper2)]
        elif upper1 == upper2:
            result1 = [(lower1,upper1)]
            result2 = [(lower2,lower1),(lower1, upper2)]
        elif upper1 > upper2:
            result1 = [(lower1,upper2),(upper2, upper1)]
            result2 = [(lower2,lower1),(lower1, upper2)]
    elif lower1 == lower2:
        if upper1 < upper2:
            result1 = [(lower1,upper1)]
            result2 = [(lower2,lower1),(lower1, upper2)]
        elif upper1 == upper2:
            result1 = [(lower1,upper1)]
            result2 = [(lower2,upper2)]
        elif upper1 > upper2:
            result1 = [(lower1,upper2), (upper2,upper1)]
            result2 = [(lower2,upper2)]
    return (result1, result2)

In [ ]:
assert de_overlap_ranges(1, 5, 2, 4) == ([(1,2),(2,2),(2,4),(4,4),(4,5),(5,5)],[(2,4)])
assert de_overlap_ranges(1, 3, 2, 4) == ([(1,2),(2,3)],[(2,3),(3,4)])
assert de_overlap_ranges(-1,3,3,5) == ([(-1,3)],[(3,5)])

assert de_overlap_ranges(2, 4,1,5) == ([(2,4)],[(1,2),(2,4),(4,5)])
assert de_overlap_ranges(2,4,1,3) == ([(2,3),(3,4)], [(1,2),(2,3)])
assert de_overlap_ranges(3,5,-1,3) == ([(3,5)], [(-1,3)])

TODO: What if the width IS already 1? Do not duplicate those!

Now we need a smart way of generating the extended list of ranges while de-overlapping them.  
1. Initialize this extended list.  
2. Pop the first element in the original list.
3. Compare it to all elements in the current extended list that have the same orientation (Horizontal / Vertical).
4. If an element in the extended list is split, put all splits in place and continue afterwards.
5. In the current element from the original list is split, keep the first one and put the second on top of the original list (so that it is processed next).

In [90]:
iwc_copy = deepcopy(instructions_with_coordinates)
extended_instructions_with_coordinates = []
while iwc_copy:
#for _ in range(5):
    direction, start, end = iwc_copy.pop(0)
    for i in range(len(extended_instructions_with_coordinates)):
        direction_other, start_other, end_other = extended_instructions_with_coordinates[i]
        if (direction in ['R', 'L'] and direction_other in ['R', 'L']) or (direction in ['U', 'D'] and direction_other in ['U', 'D']):
            extended1, extended2 = de_overlap_ranges(start, end, start_other, end_other)
            extended1 = [(direction, start, end) for (start, end) in extended1]
            extended2 = [(direction_other, start, end) for (start, end) in extended2]
            if len(extended1) > 1:
                if direction in ['U','L']:
                    extended1 = extended1[::-1]
                iwc_copy = extended1[1:] + iwc_copy
                _, start, end = extended1[0]
            if len(extended2) > 1:
                extended_instructions_with_coordinates = extended_instructions_with_coordinates[:i] + extended2[::1 if direction_other in ['R','D'] else -1] + extended_instructions_with_coordinates[i+1:]
    extended_instructions_with_coordinates.append((direction, start, end))
    print(f'iwc_copy: {len(iwc_copy)}, extended_iwc: {len(extended_instructions_with_coordinates)}')

iwc_copy: 13, extended_iwc: 1
iwc_copy: 12, extended_iwc: 2
iwc_copy: 11, extended_iwc: 3
iwc_copy: 10, extended_iwc: 4
iwc_copy: 9, extended_iwc: 5
iwc_copy: 8, extended_iwc: 6
iwc_copy: 8, extended_iwc: 8
iwc_copy: 7, extended_iwc: 9
iwc_copy: 7, extended_iwc: 11
iwc_copy: 6, extended_iwc: 12
iwc_copy: 5, extended_iwc: 14
iwc_copy: 5, extended_iwc: 15
iwc_copy: 4, extended_iwc: 16
iwc_copy: 4, extended_iwc: 18
iwc_copy: 3, extended_iwc: 19
iwc_copy: 3, extended_iwc: 21
iwc_copy: 2, extended_iwc: 24
iwc_copy: 1, extended_iwc: 25
iwc_copy: 2, extended_iwc: 26
iwc_copy: 1, extended_iwc: 27
iwc_copy: 0, extended_iwc: 28


For the real input, it's stuck in an infinity loop at the 13th instruction. Investigate later...

Now construct the condensed trench map:

In [91]:
extended_instructions = [((end-start), direction) for (direction, start, end) in extended_instructions_with_coordinates]

In [92]:
#Estimate size of trench
n_right = sum(direction == 'R' for _, direction in extended_instructions)
n_down = sum(direction == 'D' for _, direction in extended_instructions)
n_left = sum(direction == 'L' for _, direction in extended_instructions)
n_up = sum(direction == 'U' for _, direction in extended_instructions)
n_right, n_down, n_left, n_up

(6, 8, 6, 8)

In [93]:
#Properly estimate size of trench map
i,j = 0,0
max_right, max_left, max_up, max_down = 0,0,0,0
for _, direction in extended_instructions:
    if direction == 'R':
        j += 1
        max_right = max(max_right, j)
    elif direction == 'L':
        j -= 1
        max_left = min(max_left, j)
    elif direction == 'D':
        i += 1
        max_down = max(max_down, i)
    elif direction == 'U':
        i -= 1
        max_up = min(max_up, i)
#Left and Up will be negative
max_left *= -1
max_up *= -1

In [94]:
max_right, max_left, max_up, max_down

(6, 0, 0, 5)

In [95]:
#Initialize trench map
condensed_trench_map = np.full((max_up + max_down+1, max_left + max_right+1), fill_value='.')
#Initialize lists of horizontal and vertical sizes of each trench
horizontal_trench_sizes = np.zeros(max_right + max_left+1, dtype=int)
vertical_trench_sizes = np.zeros(max_up + max_down+1, dtype=int)

In [96]:
#Fill trench map
j,i = max_left, max_up
condensed_trench_map[i,j] = '#'
for distance, direction in extended_instructions:
    if direction == 'R':
        j+=1
        horizontal_trench_sizes[j] = distance
    elif direction == 'D':
        i+=1
        vertical_trench_sizes[i] = distance
    elif direction == 'L':
        horizontal_trench_sizes[j] = distance
        j-=1
    elif direction == 'U':
        vertical_trench_sizes[i] = distance
        i-=1
    condensed_trench_map[i,j] = '#'

In [97]:
condensed_trench_map

array([['#', '#', '#', '.', '.', '.', '.'],
       ['#', '.', '#', '#', '#', '#', '.'],
       ['#', '.', '.', '#', '#', '#', '.'],
       ['#', '#', '.', '#', '#', '#', '.'],
       ['.', '#', '.', '#', '#', '#', '#'],
       ['.', '#', '#', '#', '#', '#', '#']], dtype='<U1')

In [98]:
extended_instructions

[(5411, 'R'),
 (456526, 'R'),
 (56407, 'D'),
 (35119, 'R'),
 (112010, 'R'),
 (209542, 'R'),
 (299946, 'D'),
 (143901, 'D'),
 (419393, 'D'),
 (367720, 'R'),
 (266681, 'D'),
 (367720, 'L'),
 (209542, 'L'),
 (266681, 'U'),
 (419393, 'U'),
 (143901, 'U'),
 (112010, 'L'),
 (143901, 'D'),
 (419393, 'D'),
 (266681, 'D'),
 (35119, 'L'),
 (456526, 'L'),
 (266681, 'U'),
 (419393, 'U'),
 (5411, 'L'),
 (143901, 'U'),
 (299946, 'U'),
 (56407, 'U')]

Fill it. Need a different algorithm than in part 1 because of the condensedness...

Iterate over tiles with '.'. Check if there is a '#' in every direction. If yes, color it.

In [99]:
condensed_trench_map_filled = condensed_trench_map.copy()
for i,j in zip(*np.where(condensed_trench_map_filled == '.')):
    if any(condensed_trench_map_filled[:i,j] == '#') and \
    any(condensed_trench_map_filled[i:,j] == '#') and \
    any(condensed_trench_map_filled[i,:j] == '#') and \
    any(condensed_trench_map_filled[i,j:] == '#'):
        condensed_trench_map_filled[i,j] = '#'

In [100]:
condensed_trench_map_filled

array([['#', '#', '#', '.', '.', '.', '.'],
       ['#', '#', '#', '#', '#', '#', '.'],
       ['#', '#', '#', '#', '#', '#', '.'],
       ['#', '#', '#', '#', '#', '#', '.'],
       ['.', '#', '#', '#', '#', '#', '#'],
       ['.', '#', '#', '#', '#', '#', '#']], dtype='<U1')

In [101]:
horizontal_trench_sizes

array([     0,   5411, 456526,  35119, 112010, 209542, 367720])

In [102]:
vertical_trench_sizes

array([     0,  56407, 299946, 143901, 419393, 266681])

Now the result is the sum of all areas of colored tiles.

In [105]:
res = 0
for i,j in zip(*np.where(condensed_trench_map_filled == '#')):
    res += int(horizontal_trench_sizes[j]+1) * int(vertical_trench_sizes[i]+1)

In [106]:
res

1223433131122